In [3]:
pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/165.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/165.5 kB ? eta -:--:--
     ------ ------------------------------ 30.7/165.5 kB 660.6 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/165.5 kB 919.0 kB/s eta 0:00:01
     ---------------------------- --------- 122.9/165.5 kB 1.0 MB/s eta 0:00:01
     ------------------------------------ 165.5/165.5 kB 995.2 kB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.0 MB 3.2 MB/s eta 0:00:03
    --------------------------------------- 0.1/8.0 MB 1.8 MB/s eta 0:00:05
    --------------------------------------- 0.2/8.0 MB 1.7 MB/s eta 0:00:05
   - -------------------------------------- 0.2/8.0 MB 1.8 MB/s eta 0:00:05
   - -------------------------

In [28]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
path = "../data/Challenge/popularity/docs.tsv"
with open(path,'r', encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip().split('\t')
        print(line)
        break

['N55528', 'lifestyle', 'lifestyleroyals', 'The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By', "Shop the notebooks, jackets, and more that the royals can't live without.", 'https://assets.msn.com/labs/mind/AAGH0ET.html', '[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]', '[]']


In [29]:
data_root_path = "../data/Challenge/"
embedding_path = None
KG_root_path = None
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title','vert','entity'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
             'max_entity_num':5}

In [30]:
News = NewsContent(config)
TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

19779
2472
2473


In [31]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

19779
22179
2473
2472


In [ ]:
# title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)

In [ ]:
train_generator = TrainGenerator(News,TrainUsers,train_sess,train_user_id,train_buckets,train_label,32)
test_user_generator = UserGenerator(News,TestUsers,32)
val_user_generator = UserGenerator(News,ValidUsers,32)
news_generator = NewsGenerator(News,32)

In [ ]:
# for i in range(10):
#     model_config = {
#         'news_encoder':1,
#         'popularity_user_modeling':True,
#         'rel':True,
#         'ctr':True,
#         'content':True,
#         'rece_emb':True,
#         'activity':True
# 
#     }
# 
#     model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = create_pe_model(config,model_config,News,title_word_embedding_matrix,News.entity_embedding)
#     model.fit_generator(train_generator,epochs=2)
#     news_scoring = news_encoder.predict_generator(news_generator,verbose=True)
#     user_scoring = user_encoder.predict_generator(test_user_generator,verbose=True)
#     val_user_scoring = user_encoder.predict_generator(val_user_generator,verbose=True)
# 
# 
#     news_bias_vecs = bias_news_encoder.predict_generator(news_generator,verbose=True)
# 
#     if model_config['content'] and not model_config['rece_emb']:
#         bias_candidate_score = bias_content_scorer.predict(news_bias_vecs,batch_size=32,verbose=True)
#         bias_candidate_score = bias_candidate_score[:,0]
#     else:
#         bias_candidate_score = 0
# 
#     ctr_weight = scaler.get_weights()[0][0,0]
#     time_embedding_matrix = time_embedding_layer.get_weights()[0]
#     
#     predicted_activity_gates = activity_gater.predict(user_scoring,verbose=True)
#     predicted_activity_gates = predicted_activity_gates[:,0]
#     
#     val_predicted_activity_gates = activity_gater.predict(val_user_scoring,verbose=True)
#     val_predicted_activity_gates = val_predicted_activity_gates[:,0]
#     
#     rankings = news_ranking(model_config,ctr_weight,predicted_activity_gates,user_scoring,news_scoring,
#                                 bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                 News,test_impressions)
#     
#     val_rankings = news_ranking(model_config,ctr_weight,val_predicted_activity_gates,val_user_scoring,news_scoring,
#                                bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                News,val_impressions)
#     
#     performance = evaluate_performance(rankings,test_impressions)
#     val_performance = evaluate_performance(val_rankings,val_impressions)
# 
#     cold = []
#     for TOP_COLD_NUM in [0,1,3,5,]:
#         g = evaluate_cold_users(rankings,test_impressions,TestUsers.click,TOP_COLD_NUM)
#         cold.append(g)
#     diversity = []
#     for TOP_DIVERSITY_NUM in range(1,11):
#         div_top = evaluate_diversity_topic_all(TOP_DIVERSITY_NUM,rankings,test_impressions,News,TestUsers)
#         div_ilxd = evaluate_density_ILxD(TOP_DIVERSITY_NUM,rankings,test_impressions,news_scoring)
#         diversity.append([div_top,div_ilxd])
